In [5]:
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score # Please note that this is the only sklearn function that can be utilized.

# Load data

In [6]:
# Load the train/val/test dataset

df_train = pd.DataFrame(pd.read_csv("./PR_HW3_Train.csv"))
df_val   = pd.DataFrame(pd.read_csv("./PR_HW3_Val.csv"))
df_test  = pd.DataFrame(pd.read_csv("./PR_HW3_Test.csv"))
#df_test  = pd.DataFrame(pd.read_csv("./PR_HW3_Test_Keep.csv"))

X_train = df_train[['Feature1', 'Feature2', 'Feature3', 'Feature4', 'Feature5', 'Feature6', 'Feature7']].to_numpy()
y_train = df_train["Target"].to_numpy()

X_val = df_val[['Feature1', 'Feature2', 'Feature3', 'Feature4', 'Feature5', 'Feature6', 'Feature7']].to_numpy()
y_val = df_val["Target"].to_numpy()

X_test = df_test[['Feature1', 'Feature2', 'Feature3', 'Feature4', 'Feature5', 'Feature6', 'Feature7']].to_numpy()
y_test = df_test["Target"].to_numpy()

In [7]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(800, 7)
(800,)
(800, 7)
(800,)
(800, 7)
(800,)


# Model Implementation

In [8]:
def gini(sequence):
    classes, counts = np.unique(sequence, return_counts=True)
    impurity = 1.0
    for count in counts:
        prob = count / len(sequence)
        impurity -= prob ** 2
    return impurity

def entropy(sequence):
    classes, counts = np.unique(sequence, return_counts=True)
    impurity = 0.0
    for count in counts:
        prob = count / len(sequence)
        impurity -= prob * np.log2(prob)
    return impurity

In [9]:
class Tree():
    """
        You can add/change any variables/methods to meet your need.
    """
    def __init__(self):       
        self.feature = None
        self.threshold = None
        self.left = None
        self.right = None
        self.output = None


In [10]:
class DecisionTree():
    def __init__(self, criterion='gini', max_depth=None, max_features=None):
        
        """
            You can add/change any variables/methods to meet your need.
        """

        if criterion == 'gini':
            self.criterion = gini
        elif criterion == 'entropy':
            self.criterion = entropy
        
        if max_depth is None:
            self.max_depth = 1e9
        else:
            self.max_depth = max_depth

        self.importance = {}

    def fit(self, X, y, sample_weight=None):
        def build_tree(X, y, depth, sample_weight):
            tree = Tree()
            if depth >= self.max_depth:
                tree.output = np.bincount(y, weights=sample_weight).argmax()
                return tree
            
            if len(np.unique(y)) == 1:
                tree.output = y[0]
                return tree
            
            best_feature, best_threshold = self.find_best_split(X, y, sample_weight)

            if best_feature is None or best_threshold is None:
                tree.output = np.bincount(y, weights=sample_weight).argmax()
                return tree

            tree.feature = best_feature
            tree.threshold = best_threshold

            left_mask = []
            right_mask = []
            for i in range(len(X)):
                if X[i, best_feature] <= best_threshold:
                    left_mask.append(True)
                    right_mask.append(False)
                else:
                    left_mask.append(False)
                    right_mask.append(True)

            tree.left = build_tree(X[left_mask], y[left_mask], depth + 1, sample_weight[left_mask])
            tree.right = build_tree(X[right_mask], y[right_mask], depth + 1, sample_weight[right_mask])

            return tree
        
        if sample_weight is None:
            sample_weight = np.ones(len(X)) / len(X)
        self.n_features = X.shape[1]
        self.tree = build_tree(X, y, 0, sample_weight)

    def predict(self, X):
        predictions = np.empty(X.shape[0], dtype=int)
        for i, x in enumerate(X):
            tree = self.tree
            while tree.threshold is None:
                if x[tree.feature] <= tree.threshold:
                    tree = tree.left
                else:
                    tree = tree.right
            predictions[i] = tree.output
        return predictions


    def find_best_split(self, X, y, sample_weight):
        num_features = X.shape[1]
        best_gain = -1
        best_feature = None
        best_threshold = None

        if self.max_features:
            feature_indices = np.random.choice(num_features, self.max_features, replace=False)
        else:
            feature_indices = range(num_features)

        for feature in feature_indices:
            column = X[:, feature]
            thresholds = np.unique(column)
            for threshold in thresholds:
                y_left = y[column <= threshold]
                y_right = y[column > threshold]
                sw_left = sample_weight[column <= threshold]
                sw_right = sample_weight[column > threshold]

                if len(y_left) == 0 or len(y_right) == 0:
                    continue

                gain = self.criterion(y, sample_weight) - len(y_left) / len(y) * self.criterion(y_left, sw_left) - len(y_right) / len(y) * self.criterion(y_right, sw_right)

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold
    
    def countImportance(self):
        if not hasattr(self, 'tree'):
            raise ValueError("Tree has not been trained yet.")

        importance = np.zeros(self.n_features)
        for tree in self.forest:
            imp = np.zeros(self.n_features)
            root = [tree]
            while root:
                tree = root.pop()
                if hasattr(tree, 'threshold'):
                    imp[tree.feature] += tree.n_samples
                    root.append(tree.left)
                    root.append(tree.right)
            imp /= np.sum(imp)
            importance += imp
        self.importance = importance
        return importance

In [11]:
class RandomForest():
    """
        You can add/change any variables/methods to meet your need.
    """
    def __init__(self, n_estimators=10, max_features=None, boostrap=True, criterion='gini', max_depth=None):
        
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.boostrap = boostrap
        self.criterion = criterion
        self.max_depth = max_depth
        
    def fit(self, X, y):
        pass

    def predict(self, X):
        # majority vote
        pass

# Questions for Decision Tree

In [12]:
# For Q1
ex1 = np.array(["+", "+", "+", "+", "+", "-"])
ex2 = np.array(["+", "+", "+", "-", "-", "-"])
ex3 = np.array(["+" ,"-", "-", "-", "-", "-"])

print(f"{ex1}: entropy = {entropy(ex1)}\n{ex2}: entropy = {entropy(ex2)}\n{ex3}: entropy = {entropy(ex3)}\n")
print(f"{ex1}: gini index = {gini(ex1)}\n{ex2}: gini index = {gini(ex2)}\n{ex3}: gini index = {gini(ex3)}\n")

['+' '+' '+' '+' '+' '-']: entropy = 0.6500224216483541
['+' '+' '+' '-' '-' '-']: entropy = 1.0
['+' '-' '-' '-' '-' '-']: entropy = 0.6500224216483541

['+' '+' '+' '+' '+' '-']: gini index = 0.2777777777777777
['+' '+' '+' '-' '-' '-']: gini index = 0.5
['+' '-' '-' '-' '-' '-']: gini index = 0.2777777777777777



In [13]:
# For Q2-1, validation accuracy should be higher than or equal to 0.73

np.random.seed(0) # You may adjust the seed number in all the cells

dt_depth3 = DecisionTree(criterion='gini', max_features=None, max_depth=3)
dt_depth3.fit(X_train, y_train, sample_weight=None)

acc = accuracy_score(y_val, dt_depth3.predict(X_val))

print("Q2-1 max_depth=3: ", acc)

TypeError: fit() got an unexpected keyword argument 'sample_weight'

In [ ]:
""" Do Not Modify Below """

from sklearn.tree import DecisionTreeClassifier as SK_DecisionTreeClassifier

sk_dt = SK_DecisionTreeClassifier(criterion='gini', max_depth=3)
sk_dt.fit(X_train, y_train)
sk_acc = accuracy_score(y_val, sk_dt.predict(X_val))

assert round(acc, 3) == round(sk_acc, 3), "Because the Decision Tree without any trick has a fixed answer, your accuracy should be the same as sklearn, otherwise your implementation might have some problems"

In [ ]:
# For Q2-2, validation accuracy should be higher than or equal to 0.85

np.random.seed(0)

dt_depth10 = DecisionTree(criterion='gini', max_features=None, max_depth=10)
dt_depth10.fit(X_train, y_train, sample_weight=None)

print("Q2-2 max_depth=10: ", accuracy_score(y_val,  dt_depth10.predict(X_val)))

In [ ]:
# For Q3-1, validation accuracy should be higher than or equal to 0.73

np.random.seed(0)

dt_gini = DecisionTree(criterion='gini', max_features=None, max_depth=3)
dt_gini.fit(X_train, y_train, sample_weight=None)

print("Q3-1 criterion='gini': ", accuracy_score(y_val, dt_gini.predict(X_val)))


In [ ]:
# For Q3-2, validation accuracy should be higher than or equal to 0.77

np.random.seed(0)

dt_entropy = DecisionTree(criterion='entropy', max_features=None, max_depth=3)
dt_entropy.fit(X_train, y_train, sample_weight=None)

print("Q3-2 criterion='entropy': ", accuracy_score(y_val, dt_entropy.predict(X_val)))

In [ ]:
# For Q4

# Use simply counting to get the feature importance: dt_depth10.importance

labelList=['Feature1', 'Feature2', 'Feature3', 'Feature4', 'Feature5', 'Feature6', 'Feature7']

plt.title('Feature Importance')

plt.show()

# Questions for Random Rorest

In [ ]:
# For Q6-1, validation accuracy should be higher than or equal to 0.88

np.random.seed(0)

rf_estimators10 = RandomForest(n_estimators=10, max_features=np.sqrt(X_train.shape[1]), boostrap=True, criterion='gini', max_depth=None)
rf_estimators10.fit(X_train, y_train)

print("Q6-1 n_estimators=10: ", accuracy_score(y_val, rf_estimators10.predict(X_val)))

In [ ]:
# For Q6-2, validation accuracy should be higher than or equal to 0.89

np.random.seed(0)

rf_estimators50 = RandomForest(n_estimators=50, max_features=np.sqrt(X_train.shape[1]), boostrap=True, criterion='gini', max_depth=None)
rf_estimators50.fit(X_train, y_train)

print("Q6-1 n_estimators=50: ", accuracy_score(y_val, rf_estimators50.predict(X_val)))

In [ ]:
# For Q7-1, validation accuracy should be higher than or equal to 0.88

np.random.seed(0)

rf_maxfeature_sqrt = RandomForest(n_estimators=10, max_features=np.sqrt(X_train.shape[1]), boostrap=True, criterion='gini', max_depth=None)
rf_maxfeature_sqrt.fit(X_train, y_train)

print("Q7-1 max_features='sqrt': ", accuracy_score(y_val,  rf_maxfeature_sqrt.predict(X_val)))

In [ ]:
# For Q7-2, validation accuracy should be higher than or equal to 0.86

np.random.seed(0)

rf_maxfeature_none = RandomForest(n_estimators=10, max_features=None, boostrap=True, criterion='gini', max_depth=None)
rf_maxfeature_none.fit(X_train, y_train)

print("Q7-1 max_features='All': ", accuracy_score(y_val, rf_maxfeature_none.predict(X_val)))

# Train your own model

In [ ]:
# Build and train your model

In [ ]:
test_pred = your_model.predict(X_test)

print("test_pred shape: ", test_pred.shape)

In [ ]:
# output csv
df_test = pd.DataFrame(pd.read_csv("./PR_HW3_Test.csv"))
df_test["Target"] = test_pred
df_test.to_csv("sample_output.csv")